In [3]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display

# Pathfinder Playtest Encounter Creator

## Overview
This is a python based tool for DMs to design encounters for the Pathfinder Playtest Rulset. The goal is to be able to quickly and efficiently design encounters with minimal input from the user.

___

## Plan and Implementation
To achieve this goal, my approach is to use stepwise design in an implementation similar to the "rod-cutting" dynamic programming problem where all costs for each "length" are equal. That is, the minimum viable product goal is to naievely find all possible permutations of creatures that match our parameters.

The algorithm should take several parameters. 
- Party Level
- Party Size
- Encounter Severirty
- (Optional) The maximum number of creatures for the encounter.
- (Optional) The minimum number of creatures for the encounter.
- (Optional) An XP Budget

### The Algorithm
The algorithm itself will try to attain a combination of creatures that is equal to or under the cost of the encounter severity OR an XP budget if provided. 
To do this it will use top-down greedy programming by first choosing the largest XP cost creature that fits the budget. Once found it will store that local solution into a database and then recurse finding the next highest costing creature that fits. When a creature is found and there is still a remaning budget left, it will then use that new remaining budget and recurse down again choosing the local maximum until the base case where the remaining budget is 0.

### Stretch Goals

 - __Themed Encounters__: I want the application to be able to design encounters based on themes such as "Undead" or "Jungle" or some arbitrary theme. On a napkin, I imagine this being implemented by creatures being associated with something like traits, attributes, or creature types. This allows most of the filtering to be done in the data wrangling portion of the application so that the algorithm doesn't need much changing.
 
 - __DM "Encounter Cheat Sheet"__: This originally started as a small project to make it easier for me as a DM to make an encounter on the fly. As most of we DMs know, all the planning in the world doesn't mean you're fully prepared and I wanted an quick and easy tool to create encounters in the events where it would narritively make sense when I didn't have something prepared. To that end, I'd like the application to be able to make a "cheat sheet". Essentially a quick reference sheet that I can use to reference their monster cards and maybe as an extra feature even further into the future, the ability to track their health.

___

## Project Notes

While attempting to develop the algorithm I had initially intended to design, I ran into a lot of issues where I felt like the approach didn't match what I was trying to achieve. Specifically, I modeled the problem on paper like a rod-cutting algorithm. I imagined the length of the rod that I needed to cut being the xp budget for the given encounter. Early on however I realized that the rod-cutting problem is fundamentally different than what I was trying to achieve. 

The rod-cutting algorithm's goal is to maximize how much "money" you can sell the rod for if you were to sell smaller parts of it individually. That is, the rod can be cut up into different sizes and each one of those sizes is worth a certain amount of money. The goal of the algorithm is to compare the combinations of different sized cuts to come out with a series of cuts that maximizes the money earned.

The is fundamentally different to what I am trying to acheive in two ways. The first is that by building an encounter, I am not trying to maximize anything. While the rod-cutting algorithm seeks to find a "best" solution, I am trying to find "all" or multiple solutions. The second is that the rod-cutting algorithm seeks to use as much of the "rod" as possible. The encounter designer is similar however it should allow for some leeway into allowing encounters that do not meet the full budget but meet the maximum number of creatures desired.

To that note, instead of immediately looking into optimal solutions, I created a naive approach algorithm that seeks to produce what I want. The way the naive algorithm works is that it loops through all mosnters in the bestiary. For every monster, it attempts to build an encounter by first putting that monster into an encounter if it is within the xp budget and calculates the remaining budget. From there it will then loop through the monster list again to put the next monster that it comes across that is within budget into the encoutner and recalculates the remaining budget. It will do this until it either does not find a monster, or the remaning budget is 0. In that case it will "publish" the encounter to a _global_ list of _possible_ encounters and delete the last monster that made no other combinations possible from the current encounter. It will then continue onto the next monster that meets requirements and repeats ad nauseum. 

This approach is significantly inefficient since the effiency is in an O(n<sup>c</sup>) class. In this case the n is the bestiary size and the c refers to the maximum amount of monsters you want in the encounter.

---
### Improvements

For my next version, I want to try figure out what ways I can make the algorithm more efficient. Specifically, the things I want to look at are limiting the search space by doing things such as:

- Stopping after finding X amount of solutions.
- In addition to that trying random combinations of monsters so that we potentially search for < N amount of monsters. 
- Using filtering for thigns such as traits, monster type, locations, etc to narrow down the size of N.
- Implementing arbitrary rules such as "First monster must meet at least 50% of total cost."
- Using a stop condition in cases where no matches were found after looping through the whole list of possible monsters. 
- Changing the base case so that if the amount of experience left is < the lowest possible xp value the algorithm stops.
- Options to dictate specific monsters that must be included.

For a more complex way to increase efficiency, I want to look at way the algorithm could be multi-threaded so that two different possibilities of encouners can be built at the same time. Or, while one monster is being found, another thread could be finding the next monster with an estimated remaining budget. Due to the way multithreading works in python however, I believe there wwould be some potential difficulty in maintaining data integrity and ensuring there are no race conditions.

In [4]:
#__________This section contains all of the background setup__________

# Load the bestiary database
bestiary = pd.read_csv('data/bestiary.csv');
#bestiary

# Set the baseline encounter xp values.
BUDGETS = {'Trivial': [40,10], 'Low': [60,15], 'High': [80,20], 'Severe': [120,30], 'Extreme': [160,40]}

In [5]:
#__________This section contains the Algorithm that powers the encounter builder__________

# This global variable is the master list that contains all of the encounters.
encounter_list = []

# This function does most of the heavy lifting.
# Input:
#     catalogue: The bestiary an all of their associated xp values.
#            xp: The experience budget to dictate how many monsters can fit in a given encounter.
#   party_level: The party's level. Used in finding the xp value of a creature relative to the party's level.
#         max_c: The maxmimum amount of creatures allowed in an encounter.
#         min_c: The minimum amount of creatures allowed in an encounter.
#     encounter: (Optional) This contains the current encounter being worked on. Initially starts as empty but
#                           is filled during recursive calls.
# Returns:
#          None
# This function takes a series of parameters to build an encounter and append it to a global list of possible encounters.
# Using recursion, it finds creatures in the bestiary that contain xp values less than the budget and adds it as a leaf
# node in the encounter tree. Then it attempts to recurse and add a new leaf node if there is more xp budget and there 
# is enough space left in the encounter. If neither of those conditions are met, it publishes the current encounter to the 
# global list an removes the leaf node to try a new one.
def find_creature(catalogue, xp, party_level,max_c,min_c,encounter=[]):
    
    # Base Case
    # The xp budget is 0 therefore no new leaf nodes can be added. Immediately publish the current encounter.
    if xp < 10:
        publish(encounter.copy())
    
    # Otherwise iterate through the whole list of monsters to find appropriate children
    else:
        # Loop through every monster
        for index, row in catalogue.iterrows():
            # Check to see if we can afford the xp cost of the current monster.
            if int(row[str(party_level)]) <= xp:
                encounter.append(row['CREATURE NAME'])       # We can afford it so add it as a leaf node to the encounter tree.
                newBudget = xp - int(row[str(party_level)])  # Calcualte the remaining budget.
                if len(encounter) < max_c:                   # If we still have space in the encounter, recursively call and
                                                             # attempt to find a new monster that we can afford.
                    find_creature(catalogue, newBudget, party_level, max_c, min_c,encounter)
                    
                # Once we have used up the budget we can publish.
                # This works because if we haven't added anything to the encounter then we can publish what we have.
                # If it did add something to the encounter however, that level's recursive call will handle the publishing.
                
                if len(encounter) >= min_c:     # If the encounter meets the minimum monster count we can publish.
                    publish(encounter.copy())
                del encounter[-1]               # After publishing the encounter we want to remove the leaf that was used and
                                                # so that we can try a new one.

                    
# This is a helper function to publish an encounter to the encounter list.
# Input:
#   current_enc: Contains the encounter to be published in list form.
# Returns:
#          None
def publish(current_enc):
    global encounter_list
    if len(current_enc) > 0:     # Check to make sure we aren't publishing an empty list.
        if current_enc not in encounter_list:    # Also make sure that we don't publish something that's already there.
            encounter_list.append(current_enc)
    
# This is the driver function that's called from the main body. 
# Input:
#   party_level: The party's level. Used in finding the xp value of a creature relative to the party's level.
#    party_size: The size of the party. Used in conjunction with the severity to calculate the XP budget.
#      severity: Used with the party size to calculate the XP budget in the event a budget is not provided.
#         max_c: The maxmimum amount of creatures allowed in an encounter.
#         min_c: The minimum amount of creatures allowed in an encounter.
#        budget: A manually input XP budget.
# Returns:
#          None
def build_encounters(party_level,party_size,severity,max_c,min_c,budget):
    global encounter_list
    # First, calcualte total encounter budget.
    if budget > 0:
        XP_Budget = budget
    else:
        if party_size > 4:
            XP_Budget = BUDGETS[severity][0] + (BUDGETS[severity][1] * (party_size - 4))
        elif party_size < 4:
            XP_Budget = BUDGETS[severity][0] - (BUDGETS[severity][1] * (4-party_size))
        else:
            XP_Budget = BUDGETS[severity][0]
    
    # Next, consult the database and get the XP values for each creature based on the specified level.
    xp_costs = bestiary[['CREATURE NAME', str(party_level)]]           # Get the cost for all creatures.
    nontrivial_xp_costs = xp_costs[xp_costs[str(party_level)] != '-']  # Remove all creatures of trivial difficulty.
    pruned_xp_costs = nontrivial_xp_costs[nontrivial_xp_costs[str(party_level)] != 'X'] # Remove all creatures of impossible difficulty.
    
    # Now that we have the creatures and how much they cost, build all possible encounters.
    find_creature(pruned_xp_costs,XP_Budget,party_level,max_c,min_c) # This function does most of the heavy lifting
                                                                     # and builds the encounter list into a global variable.
    temp_enc_list = encounter_list.copy() # Copy the encounters list into a temporary value to return
    encounter_list = []                   # Scrub the global encounters list clean so that it can be ready for another
                                          # query.
    return temp_enc_list                  # Return the copy of the list.


#Testing Function
#Build an High threat encounter for a party of 4 level 1 adventurers with a max of five monsters and a minimum of one.
#XP Budget: 80
#test = build_encounters(1,4,'High',5,1,0)


#encounter_list
#print('Making sure the Encounter List is now empty.',encounter_list)
#test

In [6]:
#__________This section contains the code that drives the interface__________

button = widgets.Button(description="Build Encounters!")
def on_button_clicked(b):
    print('Working...')
    solution = build_encounters(party_level.value, 
                                party_size.value, 
                                severity.value,
                                creature_count.value[1], 
                                creature_count.value[0],
                                custom_budget.value)
    
    filename = 'PL-'+str(party_level.value)+'_PS-'+str(party_size.value)+'_SEV-'+severity.value
    filename = filename+'_MIN-'+str(creature_count.value[0])+'_Max-'+str(creature_count.value[1])
    filename = filename+'_CB-'+str(custom_budget.value)+'.csv'
    
    df = pd.DataFrame(solution)
    #print("Number of Columns: ",len(df.columns))
    columnNames = []
    for col in df.columns:
        columnNames.append('Monster ' + str(col + 1))
    df.columns = columnNames
    df.to_csv(filename, index=False)
    print('Encounters saved to file: ',filename)
    
button.on_click(on_button_clicked)

party_level = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    description='Party Level:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)
party_size = widgets.IntSlider(
    value=4,
    min=0,
    max=10,
    description='Party Size:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

severity = widgets.Dropdown(
    options=['Trivial', 'Low', 'High', 'Severe', 'Extreme'],
    value='Trivial',
    description='Encounter Severity: ',
    disabled=False,
    style = {'description_width': 'initial',}
)

custom_budget = widgets.BoundedIntText(
    value='0',
    min=0,
    max=500,
    step=5,
    description='Manual XP Budget (Optional):',
    disabled=False,
    style = {'description_width': 'initial'}
)

creature_count = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=20,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='40%')
)


display(party_level)
display(party_size)
display(severity)
display(custom_budget)

print('')
print('')
print("Select a minimum and maximum number of creatures to use in the encounter:")
display(creature_count)




display(button)

IntSlider(value=1, description='Party Level:', layout=Layout(width='40%'), max=20, min=1, style=SliderStyle(de…

IntSlider(value=4, description='Party Size:', layout=Layout(width='40%'), max=10, style=SliderStyle(descriptio…

Dropdown(description='Encounter Severity: ', options=('Trivial', 'Low', 'High', 'Severe', 'Extreme'), style=De…

BoundedIntText(value=0, description='Manual XP Budget (Optional):', max=500, step=5, style=DescriptionStyle(de…



Select a minimum and maximum number of creatures to use in the encounter:


IntRangeSlider(value=(1, 20), continuous_update=False, layout=Layout(width='40%'), max=20, min=1)

Button(description='Build Encounters!', style=ButtonStyle())

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QIcon


class Example(QWidget):
    
    def __init__(self):
        super().__init__()
        
        self.initUI()
        
        
    def initUI(self):
        
        self.setGeometry(300, 300, 1080, 720)
        self.setWindowTitle('Icon')
        self.setWindowIcon(QIcon('web.png'))        
    
        self.show()
        
        
if __name__ == '__main__':
    
    app = QApplication(sys.argv)
    ex = Example()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Rachel Yasaka\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
